In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install beautifulsoup4 unidecode

In [ ]:
from bs4 import BeautifulSoup
import re
import unidecode

# Function to remove HTML tags and convert to lowercase
def clean_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()  # Remove HTML tags
    text = unidecode.unidecode(text)  # Convert Unicode characters to ASCII
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespaces
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    text = text.lower().strip()  # Convert to lowercase and strip
    return text

# Main text preprocessing function
def text_preprocess(text):
    text = clean_text(text)
    # Additional steps like acronym normalization or word segmentation can be added here if needed
    return text

In [ ]:
import pandas as pd

# Assuming bigdata_selected is already loaded from Google Drive
# Replace this with your actual data loading code if necessary
file_path = '/content/drive/My Drive/bigdata_selected.csv'
bigdata_selected = pd.read_csv(file_path)

# Define features (X) and target (y)
X = bigdata_selected['Review'].astype(str).apply(text_preprocess) # Assuming 'Review' is your feature column
y = bigdata_selected[['Taste', 'Smell', 'Quality']]  # Assuming these are your target columns


split_index = int(0.8 * len(X))
X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y[:split_index], y[split_index:]


# Print shapes to verify split
print("Training set:")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print("Validation set:")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

<ipython-input-5-ad48189212e9>:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()  # Remove HTML tags


Training set:
X_train shape: (2467,), y_train shape: (2467, 3)
Validation set:
X_val shape: (617,), y_val shape: (617, 3)


#Standard data

In [ ]:
import numpy as np
def make_outputs(df):
    outputs = []
    for row in range(len(df)):
        row_one_hot = []
        for col in range(0, len(df.columns)):
            sentiment = df.iloc[row, col]
            if   sentiment == 0: one_hot = [1, 0, 0, 0] # None
            elif sentiment == 1: one_hot = [0, 1, 0, 0] # Pos
            elif sentiment == 2: one_hot = [0, 0, 1, 0] # Neg
            elif sentiment == 3: one_hot = [0, 0, 0, 1] # Neu
            row_one_hot.extend(one_hot)

        outputs.append(row_one_hot)
    return np.array(outputs, dtype='uint8')

In [ ]:
# cái này thử sau

# import numpy as np
# def make_outputs(df):
#     outputs = []
#     for row in range(len(df)):
#         row_one_hot = []
#         for col in range(0, len(df.columns)):
#             sentiment = df.iloc[row, col]
#             if   sentiment == 0: one_hot = [1, 0, 0, 0] # None
#             elif sentiment == 1: one_hot = [0, 1, 0, 0] # Pos
#             elif sentiment == 2: one_hot = [0, 0, 1, 0] # Neg
#             elif sentiment == 3: one_hot = [0, 0, 0, 1] # Neu
#             row_one_hot.append(one_hot)

#         outputs.append(row_one_hot)
#     return np.array(outputs, dtype='uint8')

In [ ]:
y_train_standardize_output = make_outputs(y_train)
y_val_standardize_output = make_outputs(y_val)

print('Train outputs:', y_train_standardize_output.shape)
print('Validate outputs:', y_val_standardize_output.shape)
y_train_standardize_output[0]

Train outputs: (2467, 12)
Validate outputs: (617, 12)


array([0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], dtype=uint8)

#Training or fine tuning the data

### embeding

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
# import tensorflow_text as text
from tensorflow.keras.optimizers import Adam

embeding có thể đổi được

In [ ]:


# Define a Keras layer that uses the Universal Sentence Encoder
hub_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                           input_shape=[],
                           dtype=tf.string,
                           trainable=True)

# Create a Sequential model
model = tf.keras.Sequential()

# Add the USE layer to the model
model.add(hub_layer)

# Add dense layers
model.add(tf.keras.layers.Dense(32, activation="relu"))
model.add(tf.keras.layers.Dense(16, activation="relu"))
model.add(tf.keras.layers.Dense(12, activation="sigmoid"))

# Print the model summary
model.summary()
adam = Adam(learning_rate =0.001)
model.compile(optimizer=adam,
              loss='binary_crossentropy',
              metrics=tf.keras.metrics.BinaryAccuracy( name="binary_accuracy", dtype= None, threshold=0.5))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               256797824 
                                                                 
 dense (Dense)               (None, 32)                16416     
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 12)                204       
                                                                 
Total params: 256814972 (979.67 MB)
Trainable params: 256814972 (979.67 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
callback_model= tf.keras.callbacks.ModelCheckpoint('model_google_embedding.h5', monitor='val_loss')
history= model.fit(x=X_train, y =y_train_standardize_output,  epochs =10, validation_data=[X_val,y_val_standardize_output], callbacks=[callback_model])

Epoch 1/10
78/78 [==============================] - ETA: 0s - loss: 0.4696 - binary_accuracy: 0.8095

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


78/78 [==============================] - 780s 10s/step - loss: 0.4696 - binary_accuracy: 0.8095 - val_loss: 0.4231 - val_binary_accuracy: 0.8036
Epoch 2/10
78/78 [==============================] - 805s 10s/step - loss: 0.3915 - binary_accuracy: 0.8264 - val_loss: 0.4195 - val_binary_accuracy: 0.7909
Epoch 3/10
78/78 [==============================] - 773s 10s/step - loss: 0.3739 - binary_accuracy: 0.8374 - val_loss: 0.4291 - val_binary_accuracy: 0.7959
Epoch 4/10
78/78 [==============================] - 758s 10s/step - loss: 0.2740 - binary_accuracy: 0.8861 - val_loss: 0.5060 - val_binary_accuracy: 0.7898
Epoch 5/10
78/78 [==============================] - 769s 10s/step - loss: 0.1982 - binary_accuracy: 0.9171 - val_loss: 0.5578 - val_binary_accuracy: 0.7851
Epoch 6/10
78/78 [==============================] - 713s 9s/step - loss: 0.1515 - binary_accuracy: 0.9377 - val_loss: 0.6792 - val_binary_accuracy: 0.7971
Epoch 7/10
78/78 [==============================] - 784s 10s/step - loss: 0.

In [ ]:
model.save('model_final.h5')

In [ ]:
ID của bạn:
Mật khẩu: 63919

1/1 [==============================] - 0s 50ms/step
[[1.0386888e-02 8.7038743e-01 1.6344734e-02 4.5577165e-02 2.7227065e-02
  4.6728069e-01 4.8307762e-03 2.1677765e-01 5.6454778e-04 9.3168241e-01
  1.0396725e-01 3.8112905e-02]
 [4.1812703e-02 8.9718837e-01 3.7586860e-02 8.6054578e-02 1.0593653e-01
  7.3656207e-01 1.3504137e-02 2.5684494e-01 5.9946906e-03 9.5668501e-01
  8.9454561e-02 6.1959229e-02]]
